In [53]:
import pandas as pd

In [70]:
df = pd.DataFrame()
df2024 = pd.read_csv("Services/services-2024.csv")
df2025_1 = pd.read_csv("Services/services-2025-01.csv")
df2025_2 = pd.read_csv("Services/services-2025-02.csv")
df2025_3 = pd.read_csv("Services/services-2025-03.csv")
df2025_4 = pd.read_csv("Services/services-2025-04.csv")
df2025_5 = pd.read_csv("Services/services-2025-05.csv")
df2025_6 = pd.read_csv("Services/services-2025-06.csv")
df2025_7 = pd.read_csv("Services/services-2025-07.csv")
df2025_8 = pd.read_csv("Services/services-2025-08.csv")
df2025_9 = pd.read_csv("Services/services-2025-09.csv")
df2025_10 = pd.read_csv("Services/services-2025-10.csv")

stations_to_keep = ["Bad Bentheim", 'Oldenzaal', 'Hengelo Oost', 'Hengelo', 'Borne', 'Almelo de Riet', 'Almelo', 'Wierden', 'Rijssen', 
                    'Holten', 'Deventer Colmschate', 'Deventer', 'Twello', 'Apeldoorn Osseveld', 'Apeldoorn', 'Hoevelaken', 'Amersfoort Centraal', 
                    'Barneveld Noord', 'Hengelo Gezondheidspark', 'Delden', 'Goor', 'Nijverdal', 'Enschede', 'Enschede Kennispark']
trains_to_keep = [242, 1532, 1728, 1730, 3622, 3624, 7028, 7030, 7032, 7928, 7930, 7932, 17930, 
                  20354, 20358, 31228, 31230, 31232, 31314, 31316, 31318, 31418, 31420]
dataframes_to_add = [df2024, df2025_1, df2025_2, df2025_3, df2025_4, df2025_5, df2025_6, df2025_7, df2025_8, df2025_9, df2025_10]

for i in dataframes_to_add:
    i = i[i['Service:Train number'].isin(trains_to_keep)]
    i = i[i['Stop:Station name'].isin(stations_to_keep)]
    df = pd.concat([df, i], ignore_index=True)

df.info()
df.to_csv("Services/services-peak-trains-(possible-influence-by-242).csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72632 entries, 0 to 72631
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Service:RDT-ID                72632 non-null  int64  
 1   Service:Date                  72632 non-null  object 
 2   Service:Type                  72632 non-null  object 
 3   Service:Company               72632 non-null  object 
 4   Service:Train number          72632 non-null  int64  
 5   Service:Completely cancelled  72632 non-null  bool   
 6   Service:Partly cancelled      72632 non-null  bool   
 7   Service:Maximum delay         72632 non-null  int64  
 8   Stop:RDT-ID                   72632 non-null  int64  
 9   Stop:Station code             72632 non-null  object 
 10  Stop:Station name             72632 non-null  object 
 11  Stop:Arrival time             65222 non-null  object 
 12  Stop:Arrival delay            65222 non-null  float64
 13  S

In [ ]:
delay_bentheim = {}
total_delay_bentheim = {}
delay_hengelo = {}
total_delay_hengelo = {}
delay_deventer = {}
total_delay_deventer = {}
delay_apeldoorn = {}
total_delay_apeldoorn = {}
delay_amersfoort = {}
total_delay_amersfoort = {}

def average_total_delay_per_i(total_delay_dict):
    avg_per_i = {}
    for i in range(60):
        # Alle total delays voor dit i
        delays = [total for (i_key, dag), total in total_delay_dict.items() if i_key == i]
        if delays:
            avg_per_i[i] = sum(delays) / len(delays)
        else:
            avg_per_i[i] = 0  # of None als er geen data is
    return avg_per_i

for i in range(60):
    late_rows = df[
        (df['Stop:Station name'] == 'Bad Bentheim') &
        (df['Stop:Departure delay'] == i) &
        (df['Service:Train number'] == 242)
    ]

    dagen = late_rows['Service:Date'].unique()
    df_filtered = df[df['Service:Train number'] != 242]

    for dag in dagen:
        dag_rows_bentheim = df_filtered[(df_filtered['Service:Date'] == dag) & (df_filtered['Stop:Station name'] == 'Bad Bentheim')]
        delay_bentheim[(i, dag)] = dag_rows_bentheim
        total_delay_bentheim[(i, dag)] = dag_rows_bentheim['Stop:Departure delay'].sum()
        avg_total_delay_bentheim = average_total_delay_per_i(total_delay_bentheim)

        dag_rows_hengelo = df_filtered[(df_filtered['Service:Date'] == dag) & (df_filtered['Stop:Station name'] == 'Hengelo')]
        delay_hengelo[(i, dag)] = dag_rows_hengelo
        total_delay_hengelo[(i, dag)] = dag_rows_hengelo['Stop:Departure delay'].sum()
        avg_total_delay_hengelo = average_total_delay_per_i(total_delay_hengelo)

        dag_rows_deventer = df_filtered[(df_filtered['Service:Date'] == dag) & (df_filtered['Stop:Station name'] == 'Deventer')]
        delay_deventer[(i, dag)] = dag_rows_deventer
        total_delay_deventer[(i, dag)] = dag_rows_deventer['Stop:Departure delay'].sum()
        avg_total_delay_deventer = average_total_delay_per_i(total_delay_deventer)

        dag_rows_apeldoorn = df_filtered[(df_filtered['Service:Date'] == dag) & (df_filtered['Stop:Station name'] == 'Apeldoorn')]
        delay_apeldoorn[(i, dag)] = dag_rows_apeldoorn
        total_delay_apeldoorn[(i, dag)] = dag_rows_apeldoorn['Stop:Departure delay'].sum()
        avg_total_delay_apeldoorn = average_total_delay_per_i(total_delay_apeldoorn)

        dag_rows_amersfoort = df_filtered[(df_filtered['Service:Date'] == dag) & (df_filtered['Stop:Station name'] == 'Amersfoort Centraal')]
        delay_amersfoort[(i, dag)] = dag_rows_amersfoort
        total_delay_amersfoort[(i, dag)] = dag_rows_amersfoort['Stop:Arrival delay'].sum()
        avg_total_delay_amersfoort = average_total_delay_per_i(total_delay_amersfoort)

In [74]:
df_avg_total_delays = pd.DataFrame({
    "i": range(60),
    "Bad Bentheim": [avg_total_delay_bentheim[i] for i in range(60)],
    "Hengelo": [avg_total_delay_hengelo[i] for i in range(60)],
    "Deventer": [avg_total_delay_deventer[i] for i in range(60)],
    "Apeldoorn": [avg_total_delay_apeldoorn[i] for i in range(60)],
    "Amersfoort Centraal": [avg_total_delay_amersfoort[i] for i in range(60)],
    # voeg andere stations toe
})

df_avg_total_delays.head(60)

,i,Bad Bentheim,Hengelo,Deventer,Apeldoorn,Amersfoort Centraal
0,0,3.035714,4.892857,4.350000,1.578571,2.914286
1,1,3.020833,6.958333,5.625000,3.062500,4.916667
2,2,5.117647,4.352941,4.764706,1.205882,3.058824
3,3,4.574468,4.723404,4.872340,1.212766,2.957447
4,4,2.190476,6.857143,6.595238,2.285714,6.309524
5,5,3.736842,8.368421,6.000000,1.763158,3.078947
6,6,5.027778,7.888889,6.250000,2.833333,5.277778
7,7,8.434783,8.391304,8.347826,3.652174,4.913043
8,8,6.064516,5.032258,5.387097,2.870968,4.516129
9,9,6.533333,7.000000,7.600000,3.666667,4.933333


In [ ]:
print(delay_bad_bentheim[1]['Service:Date'].unique())

['2025-02-05' '2025-03-02']


In [ ]:
df_delay = {}
delay_hengelo = {}
delay_bad_bentheim = {}
delay_deventer = {}
delay_apeldoorn = {}
delay_amersfoort = {}
average_delay_hengelo = {}
std_dev_delay_hengelo = {}
average_delay_bad_bentheim = {}
std_dev_delay_bad_bentheim = {}
average_delay_deventer = {}
std_dev_delay_deventer = {}
average_delay_apeldoorn = {}
std_dev_delay_apeldoorn = {}
average_delay_amersfoort = {}
std_dev_delay_amersfoort = {}
aantal_dagen = {}

for i in range(60):
    late_rows = df[
        (df['Stop:Station name'] == 'Bad Bentheim') &
        (df['Stop:Departure delay'] == i) &
        (df['Service:Train number'] == 242)
    ]

    dagen = late_rows['Service:Date'].unique()
    treinseries_dagen = df[df['Service:Date'].isin(dagen)]

    df_delay[i] = treinseries_dagen  # opslaan
    aantal_dagen[i] = len(dagen)
    
    delay_bad_bentheim[i] = df_delay[i][df_delay[i]['Stop:Station name'] == 'Bad Bentheim']
    average_delay_bad_bentheim[i] = delay_bad_bentheim[i]['Stop:Departure delay'].mean()
    std_dev_delay_bad_bentheim[i] = delay_bad_bentheim[i]['Stop:Departure delay'].std()

    delay_hengelo[i] = df_delay[i][df_delay[i]['Stop:Station name'] == 'Hengelo']
    average_delay_hengelo[i] = delay_hengelo[i]['Stop:Departure delay'].mean()
    std_dev_delay_hengelo[i] = delay_hengelo[i]['Stop:Departure delay'].std()

    delay_deventer[i] = df_delay[i][df_delay[i]['Stop:Station name'] == 'Deventer']
    average_delay_deventer[i] = delay_deventer[i]['Stop:Departure delay'].mean()
    std_dev_delay_deventer[i] = delay_deventer[i]['Stop:Departure delay'].std()

    delay_apeldoorn[i] = df_delay[i][df_delay[i]['Stop:Station name'] == 'Apeldoorn']
    average_delay_apeldoorn[i] = delay_apeldoorn[i]['Stop:Departure delay'].mean()
    std_dev_delay_apeldoorn[i] = delay_apeldoorn[i]['Stop:Departure delay'].std()

    delay_amersfoort[i] = df_delay[i][df_delay[i]['Stop:Station name'] == 'Amersfoort Centraal']
    average_delay_amersfoort[i] = delay_amersfoort[i]['Stop:Arrival delay'].mean()
    std_dev_delay_amersfoort[i] = delay_amersfoort[i]['Stop:Arrival delay'].std()
    

In [ ]:
print(dagen)

['2025-07-03']


In [24]:
rows = []

for i in range(60):
    rows.append({
        "delay_at_Bad_Bentheim": i,
        "aantal_dagen": aantal_dagen.get(i),

        "mean_bad_bentheim": average_delay_bad_bentheim.get(i),
        "std_bad_bentheim": std_dev_delay_bad_bentheim.get(i),
        
        "mean_hengelo": average_delay_hengelo.get(i),
        "std_hengelo": std_dev_delay_hengelo.get(i),

        "mean_deventer": average_delay_deventer.get(i),
        "std_deventer": std_dev_delay_deventer.get(i),

        "mean_apeldoorn": average_delay_apeldoorn.get(i),
        "std_apeldoorn": std_dev_delay_apeldoorn.get(i),

        "mean_amersfoort": average_delay_amersfoort.get(i),
        "std_amersfoort": std_dev_delay_amersfoort.get(i),
    })

df_stats = pd.DataFrame(rows)

In [25]:
df_stats.head(60)

,delay_at_Bad_Bentheim,aantal_dagen,mean_bad_bentheim,std_bad_bentheim,mean_hengelo,std_hengelo,mean_deventer,std_deventer,mean_apeldoorn,std_apeldoorn,mean_amersfoort,std_amersfoort
0,0,140,1.011905,2.833039,0.606933,2.798041,0.840077,3.677114,1.112821,5.292726,0.637895,3.542521
1,1,48,1.340278,2.429893,0.779141,2.607582,0.856734,2.188552,1.125828,2.628826,0.749271,2.151049
2,2,34,2.372549,3.749380,0.560345,1.781859,0.751073,1.938230,0.536082,1.109259,0.536481,2.181367
3,3,47,2.524823,3.644137,0.758475,1.410053,0.906627,2.061080,0.864662,2.109797,0.602484,2.047046
4,4,42,2.063492,2.352007,1.015660,1.900490,1.144654,2.377169,1.159420,2.485715,1.049536,3.264341
5,5,38,2.912281,2.639259,1.371007,2.505648,1.428571,3.043632,1.663793,3.137276,0.832753,2.489118
6,6,36,3.675926,3.916962,1.445783,2.988113,1.810036,3.515092,3.321429,4.948772,1.584229,3.649071
7,7,23,5.144928,5.128569,2.119522,4.969674,3.081395,6.873485,4.916667,7.189096,2.252941,5.307204
8,8,31,4.688172,4.711177,1.652568,3.763076,2.621005,5.188883,5.070588,5.931731,2.088372,4.147582
9,9,15,5.177778,4.638225,1.720930,3.822528,2.575221,4.473743,4.772727,5.806426,2.097087,4.303475
